<a href="https://colab.research.google.com/github/prasannashrestha011/Understanding-tensors/blob/main/pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

In [ ]:
##adding up column to the row,resulting tensor in one D
X = torch.tensor([[1, 2, 3], [4, 5, 6]])
X.sum(axis=1)

tensor([ 6, 15])

In [ ]:
#obtaining mean of tensor
X.sum()/X.numel()

tensor(3.5000)

In [ ]:
#non reduction form of dimension by adding up rows and cols
X.sum(axis=0,keepdim=True)

tensor([[5, 7, 9]])

In [ ]:
A=torch.tensor([[1,2],[3,4]])
A_sum=A.sum(axis=1,keepdim=True)
normalized_A=A/A_sum
normalized_A

tensor([[0.3333, 0.6667],
        [0.4286, 0.5714]])

In [ ]:
A=torch.tensor([[1,2,3],[4,5,6],[7,8,9]])
print("Row cumulative sum= ",A.cumsum(axis=0))
print("Column cumulative sum: \n ",A.cumsum(axis=1))

Row cumulative sum=  tensor([[ 1,  2,  3],
        [ 5,  7,  9],
        [12, 15, 18]])
Column cumulative sum: 
  tensor([[ 1,  3,  6],
        [ 4,  9, 15],
        [ 7, 15, 24]])
